# Tutorial: Plot Galaxy Rotation Curve with SDSS-IV MaNGA data
In this tutorial, we use [SDSS-IV MaNGA](https://www.sdss.org/surveys/manga/) data to measure line of sight velocity and plot rotation curve of galaxy. 
## Required packages
* NUMPY: v1.14.3
* SCIPY: v1.1.0
* MATPLOTLIB: v2.2.2
* ASTROPY: v3.0.4
* PPXF: v6.7.1 (included in the distribution)

In [ ]:
# import required packages
import numpy as np
from math import pi
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import cm
from time import process_time as clock
from astropy.io import fits
import urllib.request
import os.path
from ppxf_wrap import ppxf_wrap

%matplotlib inline

---
## Download MaNGA Data & Catalog
Download MaNGA cube data and catalog from SDSS-IV DR14 Science Archieve Server (https://data.sdss.org/sas/dr14/)

* MaNGA data access <https://www.sdss.org/dr14/manga/manga-data/data-access/>
* MaNGA catalog <https://www.sdss.org/dr13/manga/manga-data/catalogs/>
    * datamodel: <https://data.sdss.org/datamodel/files/MANGA_SPECTRO_REDUX/DRPVER/drpall.html>
    
We use logarithmically-binned cube data (LOGCUBE)

In [ ]:
# plateifu of galaxy to download data
plateifu='8329-6103' # Spiral Galaxy
plateifu='8137-6103' # Elliptical Galaxy


plate, ifuname=plateifu.split('-')
cdir='./'  #current directory

# MaNGA cube file
cube_file='manga-'+plateifu+'-LOGCUBE.fits.gz'   # file to de downloaded

# download MaNGA cube file if not exist.
if not os.path.isfile(cdir+cube_file):
    sas_url="https://data.sdss.org/sas/dr14/manga/spectro/redux/v2_1_2/"
    print("Start downloading "+cube_file+" from SDSS server. This may take several minutes.")
    urllib.request.urlretrieve(sas_url+plate+'/stack/'+cube_file, cdir+cube_file)
    print('Downloaded complete')


# MaNGA Catalog file (DR14)
drpall_file="drpall-v2_1_2.fits"

# download drpall file if not exist.
if not os.path.isfile(cdir+drpall_file):
    sas_url="https://data.sdss.org/sas/dr14/manga/spectro/redux/v2_1_2/"
    print("Start downloading "+drpall_file+" from SDSS server. This may take several seconds.")
    urllib.request.urlretrieve(sas_url+"drpall-v2_1_2.fits", cdir+drpall_file)    
    print('Downloaded complete')
    
image_file=plateifu+'.png'
if not os.path.isfile(cdir+image_file):
    sas_url="https://data.sdss.org/sas/dr14/manga/spectro/redux/v2_1_2/"
    print("Start downloading "+image_file+" from SDSS server. This may take several seconds.")
    urllib.request.urlretrieve(sas_url+plate+'/stack/images/'+ifuname+'.png', cdir+image_file)
    print('Downloaded complete')
    
# Show MaNGA galaxy image
fig=plt.figure(figsize=(4,4), dpi=200)
plt.imshow(mpimg.imread(cdir+image_file),aspect='equal')
plt.axis('off')

---
## Read galaxy information
In this tutorial, we need information of galaxy on
* Redshift(z)
* B/A ratio (inclination)
* Position Angle (PA)

MaNGA catalog file (drpall-v2_1_2.fits) contains above information for each galaxy.

In [ ]:
drpall=fits.open(cdir+drpall_file)
tbdata=drpall[1].data
idx=np.where(tbdata['plateifu'] == plateifu)
objra=tbdata['objra'][idx][0]
objdec=tbdata['objdec'][idx][0]
nsa_z=tbdata['nsa_z'][idx][0]
nsa_elpetro_phi=tbdata['nsa_elpetro_phi'][idx][0]
nsa_elpetro_ba=tbdata['nsa_elpetro_ba'][idx][0]

print("Galaxy: ",plateifu)
print("RA, DEC:", objra, ',',objdec)
print("Redshift: ",nsa_z)
print("Position angle (degree): ", nsa_elpetro_phi)
print("b/a ratio: ", nsa_elpetro_ba)
print('More info: http://skyserver.sdss.org/dr14/en/tools/chart/navi.aspx?ra=%0.6f&dec=%0.6f&scale=0.2' % (objra, objdec))

# print(tbdata.columns.names)

---
## Read galaxy cube data
Read and explore MaNGA Cube data.
In this tutorial, we are going to
- Plot galaxy spectrum at particular spaxel
- Plot galaxy intensity map

In [ ]:
# Open and read flux, inverse variance, and mask cube.
cube = fits.open(cdir+cube_file)
flux=np.transpose(cube['FLUX'].data, axes=(2, 1, 0))
ivar=np.transpose(cube['IVAR'].data, axes=(2, 1, 0))
mask = np.transpose(cube['MASK'].data, axes=(2, 1, 0))
wave = cube['WAVE'].data
specres = cube['SPECRES'].data

flux_header = cube['FLUX'].header

### Plot galaxy spectrum
* Plot galaxy spectrum at the galaxy center.
* User can plot other part by changing x_plot and y_plot

In [ ]:
#plot manga spectrum at the center
x_center = np.int(flux_header['CRPIX1']) - 1
y_center = np.int(flux_header['CRPIX2']) - 1

x_plot=x_center
y_plot=y_center

# User can plot other part of the galaxy
#x_plot=28
#y_plot=35

# fig=plt.figure(figsize=(9, 6))
matplotlib.rcParams['figure.figsize'] = [9, 6]
plt.plot(wave, flux[x_plot, y_plot])
plt.xlabel('$\lambda \, [\AA]$')
plt.ylabel(flux_header['BUNIT'])
plt.show()
# plt.rcParams['figure.figsize'] = [9, 6]

print('Cube 2D size: ', flux.shape[0], 'by', flux.shape[1])

### Plot galaxy intensity map around H$_{\alpha}$
* Integrate spectrum flux around H$_{\alpha}$ (6563 Angstrom) line. 
    * restframe 6550 to 6680 Angstrom
* Plot intensity map

In [ ]:
# process flux data 
do_not_use = (mask & 2**10) != 0
flux_m = np.ma.array(flux, mask=do_not_use)
ivar_m = np.ma.array(ivar, mask=do_not_use)

min_wave=6550
max_wave=6680
redshift = nsa_z
ind_wave = np.where((wave / (1 + redshift) > min_wave) & (wave / (1 + redshift) < max_wave))[0]
halpha = flux_m[:, :, ind_wave].sum(axis=2) # integrate flux along wavelength vector
im = halpha.T #Transpose numpy array

# Convert from array indices to arcsec relative to IFU center
dx = flux_header['CD1_1'] * 3600.  # deg to arcsec
dy = flux_header['CD2_2'] * 3600.  # deg to arcsec
x_extent = (np.array([0., im.shape[0]]) - (im.shape[0] - x_center)) * dx * (-1)
y_extent = (np.array([0., im.shape[1]]) - (im.shape[1] - y_center)) * dy
extent = [x_extent[0], x_extent[1], y_extent[0], y_extent[1]]

plt.imshow(im, extent=extent, cmap=cm.YlGnBu_r, vmin=0.1, vmax=100, origin='lower', interpolation='none')
plt.colorbar(label=flux_header['BUNIT'])
plt.xlabel('arcsec')
plt.ylabel('arcsec')

---
## Extract galaxy kinematics
* We use Penalized Pixel-Fitting method (pPXF) developed by Michele Cappellari to extract stellar kinematics of galaxy. 
    * http://www-astro.physics.ox.ac.uk/~mxc/software/#ppxf
    * pPXF(v6.7.1) in Python is included in this tutorial package. (ppxf.py and ppxf_util.py)
* pPXF fits template spectrum to the given galaxy spectrum to fit the galaxy kinematics.
* Four galaxy templates from MILES library is used in this tutorial.
    * Vazdekis (2010, MNRAS, 404, 1639) http://miles.iac.es/.
    * Metalicity (log(Z/Z_solar)=0), age: 1 Gyr, 2 Gyr, 8 Gyr, and 10 Gyr
        * Mun1.30Zp0.00T01.0000_iPp0.00_baseFe_linear_FWHM_2.51.fits
        * Mun1.30Zp0.00T07.9433_iPp0.00_baseFe_linear_FWHM_2.51.fits
        * Mun1.30Zp0.00T01.9953_iPp0.00_baseFe_linear_FWHM_2.51.fits
        * Mun1.30Zp0.00T10.0000_iPp0.00_baseFe_linear_FWHM_2.51.fits

* The warpper of pPXF is written separatly in ppxf_wrap.py.

### Measure line of sight velocity using ppxf_wrap
* To measure velocity in your own machine, **set run_vel_measure = True**. Otherwise, we use pre-saved result.
* ppxf_wrap measures velocity at each spaxel, one by one.
* Result will be saved in FITS format.

In [ ]:
run_vel_measure=False

oname=cdir+'manga-'+plateifu+'-LOGCUBE_MAPS.fits'

# i, j index of spaxel to show comaparison result
check_i=27
check_j=27

if run_vel_measure:
    redshift=nsa_z
    ppxf_obj=ppxf_wrap(redshift, wave, specres)
    nx, ny, _ = flux.shape
    resarr=np.zeros((nx,ny))
    flagarr=np.zeros((nx,ny),dtype=np.int16) #define flag array
    t = clock()
    check_flag=False
    print("%2s %2s %10s %10s %5s %4s" % ('i', 'j', 'Velocity', 'Dispersion', 'Chi2', 't'))
    for j in range(nx):
        for i in range(ny):
            t = clock()
            if np.median(flux_m[i,j]) > 0:
                ppxf_obj.flux=flux_m[i,j]
                ppxf_obj.ivar=ivar_m[i,j]
                
                res=ppxf_obj.run()
                if not res:
                    continue
                resarr[i,j]=res.sol[0]
                flagarr[i,j]=1
                if i == check_i & j == check_j:
                    check_flag=True
                    sflux=ppxf_obj.fflux
                    sbestfit=res.bestfit
                print("%02d %02d %10.3f %10.3f %5.2f %4.1f" % (i, j, res.sol[0], res.sol[1], res.chi2, clock()-t))

    if check_flag:
        plt.plot(ppxf_obj.lam_gal, sflux, 'b-')
        plt.plot(ppxf_obj.lam_gal, sbestfit, 'r-')
        plt.xlabel('$\lambda \, [\AA]$')
        plt.ylabel(flux_header['BUNIT'])

    print('Save velocity measurement data in to FITS file: ', oname)
    hdu = fits.PrimaryHDU()
    hdu.writeto(oname, overwrite=True)

    fits.append(oname, resarr)
    fits.append(oname, flagarr.astype(np.int16))

    append_file=fits.open(oname, mode='update')

    hdr=append_file[1].header
    hdr.set('EXTNAME','STELLAR_VEL')
    hdr=append_file[2].header
    hdr['EXTNAME']='FLAG'

    append_file.close()

---
## Read and plot 2D velocity map
* Use pre-saved file when user skipped the previous part.

In [ ]:
# read velocity map file
rfile='manga-'+plateifu+'-LOGCUBE_MAPS.fits'
if not os.path.isfile(cdir+rfile): # use pre-saved file if above one not exist.
    rfile='manga-'+plateifu+'-LOGCUBE_MAPS_SAVED.fits'
    
cube = fits.open(cdir+rfile)
velmap=cube['STELLAR_VEL'].data
nx, ny=np.int(flux_header['NAXIS1']) , np.int(flux_header['NAXIS2'])

# Plot 2D map
# Calculate velocity statistics (median and standard deviation) to re-scale map values for plot)
pvelmap=velmap.T
zero_idx=(pvelmap == 0)
pvelmap[zero_idx]=np.nan
finite_idx=np.isfinite(pvelmap)
medv, stdv, sigv=(np.median(pvelmap[finite_idx]), np.std(pvelmap[finite_idx]), 2)
pvelmap=pvelmap-medv

cmap= matplotlib.cm.RdBu_r
cmap.set_bad(color='white')
plt.imshow(pvelmap, extent=extent, cmap=cmap, vmin=(-stdv*sigv), vmax=stdv*sigv, origin='lower', interpolation='none')
plt.colorbar(label='km/s')
plt.xlabel('arcsec')
plt.ylabel('arcsec')

---
## Plot rotation curve
<img src="./gal_rot.png" alt="Drawing" style="width: 800px;"/>

* To plot rotation curve, $V(r)$, we need to perform coordinate transform of line of sight 2D velocity map into galactic coordinate system.
* A relation between $V_{LOS}$ and $V(r)$ is given as above.
* We will use known value of $V_{SYS}$, inclination angle $i$, and position angle $\phi_{0}$. All we need to do is calculating $r$ and $cos(\phi-\phi_{0})$
* How to calculate $r$ and $cos(\phi-\phi_{0})$ from $r'$ and $cos(\phi'-\phi'_{0})$ ?
    * $r\,cos(\phi-\phi_{0})=r' cos(\phi'-\phi'_{0})$ and $\phi_{0}=\phi'_{0}$
    * From tangent half-angle formula: $tan(\phi'/2)=y'/(x'+r')$
    * $tan(\phi-\phi_{0})=tan(\phi'-\phi_{0})\,cos\,i$
    * here $\phi_{0}=nsa\_elpetro\_phi+\pi/2$

In [ ]:
incl, phi0, cent_x, cent_y = np.arccos(nsa_elpetro_ba), (nsa_elpetro_phi+90)/180*pi, 0, 0

# below values from rotation model fitting (8329-6103)
# incl, phi0, cent_x, cent_y = 1.0487251 , 0.49254251+pi/2, 0.46911521, 0.45171812

# below values from rotation model fitting (8137-6103)
# incl, phi0, cent_x, cent_y = 0.81853998, 0.036205037+pi/2, 0.18230028, 0.32383320

aidx=np.arange(nx*ny)
ix=aidx % nx
iy=aidx // ny

x_obs=(ix-(nx/2-0.5))*0.5 # one pixel size correspond to 0.5 arcsec
y_obs=(iy-(ny/2-0.5))*0.5
r_obs=((x_obs-cent_x)**2+(y_obs-cent_y)**2)**0.5

tan_half_phi_obs=((y_obs-cent_y)/(x_obs-cent_x+r_obs))

phi_obs=2*np.arctan(tan_half_phi_obs)
tan_phi_gal=np.tan(phi_obs-phi0)/np.cos(incl)  # here phi_gal means phi_gal - PA_gal
cos_phi_gal=1/((1+tan_phi_gal**2)**0.5)

# calculate r_gal and v_rot
r_gal=r_obs*np.cos(phi_obs-phi0)/cos_phi_gal
v_rot=pvelmap.flatten()/cos_phi_gal/np.sin(incl)

# plot points only within +/- 20 degree from major axis
max_angle=20
major_idx=np.flatnonzero(np.abs((phi_obs-phi0)/pi*180 % 180 ) < max_angle)
r_gal=r_gal[major_idx]
v_rot=v_rot[major_idx]

ymax=np.max(v_rot[np.isfinite(np.abs(v_rot))])
xmax=np.max(r_gal[np.isfinite(np.abs(r_gal))])
ymul=1.3

plt.plot(r_gal, v_rot, 'ko')
xlim=plt.xlim()
plt.plot([-xmax, xmax], [0,0], 'r--')
plt.plot([0,0], [-ymax*ymul,ymax*ymul], 'r--')

plt.xlabel('$r_{gal}$ (arcsec)')
plt.ylabel('$V_{ROT}$ (km/s) ')
plt.ylim([-ymax*ymul,ymax*ymul])
plt.xlim(xlim)